In [64]:
from src.constants.constants import *
from src.feature_extraction.extract_features import *
from src.utils.helper import *
from src.models.evaluate import *
from src.models.train import *
from src.preprocessing.audio_cleaning import *
# from src.feature_extraction.save_features import *
import warnings

# Silence LightGBM warnings
warnings.filterwarnings("ignore")


In [65]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE



In [66]:
data = pd.read_csv('csv_files/data.csv')
features = pd.read_csv('csv_files/features.csv')
df = pd.merge(features, 
              data.drop(columns=['client_id', 'sentence', 'up_votes', 'down_votes', 'accent']), 
              left_on='voice_id', right_on='path', how='inner')
df.drop(columns=['path', 'Unnamed: 0'], inplace=True)
df.head()

,mean_mfcc_0,mean_mfcc_1,mean_mfcc_2,mean_mfcc_3,mean_mfcc_4,mean_mfcc_5,mean_mfcc_6,mean_mfcc_7,mean_mfcc_8,mean_mfcc_9,...,spectral_contrast_4,spectral_contrast_5,spectral_contrast_6,zcr,rms,pitch,voice_id,age,gender,label
0,-403.220062,99.762260,20.534065,41.544975,-7.227066,31.042414,-2.121207,-0.940516,2.747488,7.995158,...,17.295187,18.791175,52.073479,0.171368,0.034952,1656.884644,common_voice_en_181169.mp3,fifties,male,2
1,-223.813477,157.473358,11.694227,66.153183,-23.514465,0.811838,-19.525723,-8.694965,-6.325938,-10.528963,...,16.227987,21.994841,50.559168,0.114894,0.167097,1285.268188,common_voice_en_122004.mp3,twenties,male,0
2,-415.865570,126.014847,-36.185616,48.158478,-13.574215,-18.600719,-39.063217,-24.592718,-17.863024,-13.136163,...,21.419817,25.141222,50.535503,0.097565,0.054672,1702.509766,common_voice_en_10208833.mp3,fifties,female,3
3,-423.244720,198.545227,12.053857,62.119160,17.754673,-21.133663,6.490955,-1.768311,-28.847279,-15.727254,...,20.016475,19.788320,47.486808,0.041022,0.126251,820.089233,common_voice_en_18618243.mp3,twenties,male,0
4,-223.693176,74.879898,9.623554,114.356293,-12.664084,-20.089724,-38.216282,-0.664027,-42.319389,-13.502878,...,19.183877,25.547451,51.613073,0.188517,0.143542,1860.538818,common_voice_en_18075517.mp3,twenties,male,0


In [79]:
X = df.drop(columns=['label', 'gender', 'age', 'voice_id'])
y = df['age']

le = LabelEncoder()
y = le.fit_transform(y)
X_val_train, X_test, y_val_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)#, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_val_train, y_val_train, test_size=0.18, random_state=42, stratify=y_val_train)

In [68]:
# from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import SMOTE

# # Separate features and labels
# X = df.drop(columns=['label', 'gender', 'age', 'voice_id'])
# y = df['label']

# # Stratified splits to preserve class distribution
# X_val_train, X_test, y_val_train, y_test = train_test_split(
#     X, y, test_size=0.15, random_state=42, stratify=y)

# X_train, X_val, y_train, y_val = train_test_split(
#     X_val_train, y_val_train, test_size=0.18, random_state=42, stratify=y_val_train)

# # smote = SMOTE(random_state=42)
# undersample = RandomUnderSampler(random_state=42)
# X_train_res, y_train_res = undersample.fit_resample(X_train, y_train)

In [69]:
# X_train_res.shape

In [80]:
model = load_model('src/selected_model/knn_age.pkl')

In [70]:
%%time 
pipeline = get_pipeline(name='knn')
model = train_model(pipeline, X_train, y_train)
model

CPU times: user 256 ms, sys: 114 ms, total: 370 ms
Wall time: 399 ms


Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('kneighborsclassifier', KNeighborsClassifier())])

In [81]:
%%time
evaluate_multi(model, X_train, y_train, type = 0)



nTRAINING TESTING RESULTS: 
CONFUSION MATRIX:
[[20630  2481]
 [ 1480 95402]]
ACCURACY SCORE:
0.9670
CPU times: user 6min 17s, sys: 11.2 s, total: 6min 28s
Wall time: 48.6 s


In [82]:
%%time
evaluate_multi(model, X_val, y_val, type = 1)


\VALIDATION RESULTS: 
CONFUSION MATRIX:
[[ 4506   567]
 [  343 20925]]
ACCURACY SCORE:
0.9655
CPU times: user 1min 27s, sys: 2.73 s, total: 1min 30s
Wall time: 11.1 s


In [83]:
%%time
evaluate_multi(model, X_test, y_test, type = 2)



TESTING RESULTS: 
CONFUSION MATRIX:
[[ 4403   530]
 [  301 20590]]
ACCURACY SCORE:
0.9678
CPU times: user 1min 30s, sys: 2.84 s, total: 1min 33s
Wall time: 11.7 s


In [37]:
# import joblib
joblib.dump(model, 'src/selected_model/knn_label.pkl')

['src/selected_model/knn_label.pkl']

In [16]:
# model = load_model('src/selected_model/knn_base_40.pkl')